In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor

Alright buds we are on the path to reduce the rmse value as much as possible
without logging the label i got an average rmse of 2.3-2.5
reduce this rmse as much as possible
1. with logging the  the label i got the lowest being 1.5 high 3

In [2]:
import pandas as pd

def clean_data(df):
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    # Rename column '0.455' to 'Length'
    df = df.rename(columns={'0.455': 'Length'})
    # Rename column 'M' to 'Sex'
    df = df.rename(columns={'M': 'Sex'})
    # Rename column '0.365' to 'Diameter'
    df = df.rename(columns={'0.365': 'Diameter'})
    # Rename column '0.095' to 'Height'
    df = df.rename(columns={'0.095': 'Height'})
    # Rename column '0.514' to 'Weight'
    df = df.rename(columns={'0.514': 'Weight'})
    # Rename column '0.2245' to 'Shucked_weight'
    df = df.rename(columns={'0.2245': 'Shucked_weight'})
    # Rename column '0.101' to 'Viscera_weight'
    df = df.rename(columns={'0.101': 'Viscera_weight'})
    # Rename column '0.15' to 'Shell_weight'
    df = df.rename(columns={'0.15': 'Shell_weight'})
    # Rename column '15' to 'Age'
    df = df.rename(columns={'15': 'Age'})
    return df

# Loaded variable 'df' from URI: f:\Learngin-AIML\Projects\Abalone\abalone.data.csv
df = pd.read_csv(r'f:\Learngin-AIML\Projects\Abalone\abalone.data.csv')

df_clean = clean_data(df.copy())
df_clean.head()

,Sex,Length,Diameter,Height,Weight,Shucked_weight,Viscera_weight,Shell_weight,Age
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


In [3]:
df_clean

,Sex,Length,Diameter,Height,Weight,Shucked_weight,Viscera_weight,Shell_weight,Age
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,8
...,...,...,...,...,...,...,...,...,...
4171,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4172,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4173,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4174,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [4]:
y=df_clean[['Age']]

In [5]:
y

,Age
0,7
1,9
2,10
3,7
4,8
...,...
4171,11
4172,10
4173,9
4174,10


In [6]:
X=df_clean.drop("Age",axis=1)

In [ ]:
cat_encoder=OneHotEncoder(drop='first',sparse_output=False)
encoded_cat=cat_encoder.fit_transform(X[['Sex']])
encoded_cat=pd.DataFrame(encoded_cat,columns=cat_encoder.get_feature_names_out())
encoded_cat

,Sex_I,Sex_M
0,0.0,1.0
1,0.0,0.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0
...,...,...
4171,0.0,0.0
4172,0.0,1.0
4173,0.0,1.0
4174,0.0,0.0


In [24]:
X=X.join(encoded_cat)

In [27]:
X.drop('Sex',inplace=True,axis=1)

In [28]:
X

,Length,Diameter,Height,Weight,Shucked_weight,Viscera_weight,Shell_weight,Sex_I,Sex_M
0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,0.0,1.0
1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0.0,0.0
2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,0.0,1.0
3,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,1.0,0.0
4,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,1.0,0.0
...,...,...,...,...,...,...,...,...,...
4171,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0.0,0.0
4172,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,0.0,1.0
4173,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,0.0,1.0
4174,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,0.0,0.0


In [ ]:
df_clean

,Sex,Length,Diameter,Height,Weight,Shucked_weight,Viscera_weight,Shell_weight,Age
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,8
...,...,...,...,...,...,...,...,...,...
4171,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4172,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4173,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4174,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [29]:
y.skew()

Age    1.115157
dtype: float64

In [52]:
y_log=np.log1p(y)

In [53]:
y.skew()

Age    1.115157
dtype: float64

In [54]:
y_log.skew()

Age   -0.061065
dtype: float64

In [55]:
y

,Age
0,7.0
1,9.0
2,10.0
3,7.0
4,8.0
...,...
4171,11.0
4172,10.0
4173,9.0
4174,10.0


In [47]:
X

,Length,Diameter,Height,Weight,Shucked_weight,Viscera_weight,Shell_weight,Sex_I,Sex_M
0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,0.0,1.0
1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0.0,0.0
2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,0.0,1.0
3,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,1.0,0.0
4,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,1.0,0.0
...,...,...,...,...,...,...,...,...,...
4171,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0.0,0.0
4172,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,0.0,1.0
4173,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,0.0,1.0
4174,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,0.0,0.0


In [60]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [62]:
X_train_log,X_test_log,y_train_log,y_test_log=train_test_split(X,y,random_state=42,test_size=0.2)

In [58]:
y_train

,Age
4037,2.484907
1272,2.197225
3384,2.639057
3160,1.945910
3894,2.639057
...,...
3444,2.197225
466,2.639057
3092,2.484907
3772,2.302585


In [36]:
X_train

,Length,Diameter,Height,Weight,Shucked_weight,Viscera_weight,Shell_weight,Sex_I,Sex_M
4037,0.550,0.445,0.125,0.672,0.2880,0.1365,0.2100,1.0,0.0
1272,0.475,0.380,0.120,0.441,0.1785,0.0885,0.1505,1.0,0.0
3384,0.475,0.350,0.115,0.487,0.1940,0.1455,0.1250,0.0,0.0
3160,0.335,0.220,0.070,0.170,0.0760,0.0365,0.0500,0.0,0.0
3894,0.595,0.500,0.180,1.053,0.4405,0.1920,0.3900,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3444,0.495,0.400,0.145,0.578,0.2545,0.1305,0.1645,1.0,0.0
466,0.655,0.530,0.195,1.388,0.5670,0.2735,0.4100,0.0,1.0
3092,0.520,0.430,0.150,0.728,0.3020,0.1575,0.2350,0.0,0.0
3772,0.575,0.460,0.150,0.927,0.3330,0.2070,0.2985,0.0,0.0


In [50]:
y_train

,Age
4037,11.0
1272,8.0
3384,13.0
3160,6.0
3894,13.0
...,...
3444,8.0
466,13.0
3092,11.0
3772,9.0


In [66]:
ran_frst=RandomForestRegressor(random_state=42)
model_with_transformer=TransformedTargetRegressor(
    regressor=ran_frst,
    func=np.log1p,
    inverse_func=np.expm1
)

In [73]:
final_scores=-cross_val_score(model_with_transformer,X,y,cv=5,scoring='neg_root_mean_squared_error')
final_scores

array([3.18222338, 1.54862629, 2.36516447, 1.88494108, 1.91891313])

In [70]:
y_train

,Age
4037,11.0
1272,8.0
3384,13.0
3160,6.0
3894,13.0
...,...
3444,8.0
466,13.0
3092,11.0
3772,9.0


In [71]:
ran_rmse=-cross_val_score(ran_frst,X_train,np.ravel(y_train),cv=5,scoring='neg_root_mean_squared_error')
ran_rmse

array([2.37068227, 2.14222148, 2.15677685, 2.04038596, 2.17770759])